In [23]:
import pandas as pd
import os 
import nltk
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk import NaiveBayesClassifier
import sklearn 
print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 0.24.1.


In [24]:
#Importing the Dataset
df = pd.read_csv(r'C:\Users\ADMIN\OneDrive\Desktop\NLP\spam.csv')
#Taking only 2 coloumns into consideration
df = pd.DataFrame(df, columns= ['v1','v2'])
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
#renaming the Coloumns
df.columns = ['label', 'text']

In [26]:
df.head(10)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [27]:
#Adding an Extra coloumn to replace Spam & Ham into Binary 1's & 0's
df['spam'] = df['label'].map( {'spam': 1, 'ham': 0} ).astype(int)
df.head(15)

,label,text,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [28]:
#Displaying the Length of Each SMS
df['length'] = df['text'].apply(len)
df.head(10)

,label,text,spam,length
0,ham,"Go until jurong point, crazy.. Available only ...",0,111
1,ham,Ok lar... Joking wif u oni...,0,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155
3,ham,U dun say so early hor... U c already then say...,0,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1,147
6,ham,Even my brother is not like to speak with me. ...,0,77
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0,160
8,spam,WINNER!! As a valued network customer you have...,1,157
9,spam,Had your mobile 11 months or more? U R entitle...,1,154


In [29]:

#Deleting the Coloumns to avoid further Cluster
#Don't delete as Spam and length coloumns are required for further process
#    del df['length']
#    del df['spam']

In [9]:
#Defining a function to remove Punctuations!
import string 
string.punctuation

def rm_punc(text):
    new_text = ''.join([char for char in text if char not in string.punctuation])
    return new_text

In [10]:
#Adding a coloumn for messages with removed punctuations!
df['no_punc'] = df['text'].apply(lambda row : rm_punc(row))
df.head(10)

,label,text,spam,length,no_punc
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,0,29,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,0,49,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,Nah I dont think he goes to usf he lives aroun...
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1,147,FreeMsg Hey there darling its been 3 weeks now...
6,ham,Even my brother is not like to speak with me. ...,0,77,Even my brother is not like to speak with me T...
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0,160,As per your request Melle Melle Oru Minnaminun...
8,spam,WINNER!! As a valued network customer you have...,1,157,WINNER As a valued network customer you have b...
9,spam,Had your mobile 11 months or more? U R entitle...,1,154,Had your mobile 11 months or more U R entitled...


In [11]:
#Applyting Tokenization to sentences
import re

def tok(text):
    tokens = re.split('\W+',text)
    return tokens

df['tokenized_text'] = df['no_punc'].apply(lambda row: tok(row.lower()))
df.head(10)

,label,text,spam,length,no_punc,tokenized_text
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,0,29,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,0,49,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1,147,FreeMsg Hey there darling its been 3 weeks now...,"[freemsg, hey, there, darling, its, been, 3, w..."
6,ham,Even my brother is not like to speak with me. ...,0,77,Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ..."
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0,160,As per your request Melle Melle Oru Minnaminun...,"[as, per, your, request, melle, melle, oru, mi..."
8,spam,WINNER!! As a valued network customer you have...,1,157,WINNER As a valued network customer you have b...,"[winner, as, a, valued, network, customer, you..."
9,spam,Had your mobile 11 months or more? U R entitle...,1,154,Had your mobile 11 months or more U R entitled...,"[had, your, mobile, 11, months, or, more, u, r..."


In [12]:

#Removing StopWords
stopwords = nltk.corpus.stopwords.words('english')

def rm_sw(text):
    clean_text = [word for word in text if word not in stopwords]
    return clean_text

df['clean_text'] = df['tokenized_text'].apply(lambda row : rm_sw(row))
df.head(10)

,label,text,spam,length,no_punc,tokenized_text,clean_text
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,0,29,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,0,49,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1,147,FreeMsg Hey there darling its been 3 weeks now...,"[freemsg, hey, there, darling, its, been, 3, w...","[freemsg, hey, darling, 3, weeks, word, back, ..."
6,ham,Even my brother is not like to speak with me. ...,0,77,Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids..."
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0,160,As per your request Melle Melle Oru Minnaminun...,"[as, per, your, request, melle, melle, oru, mi...","[per, request, melle, melle, oru, minnaminungi..."
8,spam,WINNER!! As a valued network customer you have...,1,157,WINNER As a valued network customer you have b...,"[winner, as, a, valued, network, customer, you...","[winner, valued, network, customer, selected, ..."
9,spam,Had your mobile 11 months or more? U R entitle...,1,154,Had your mobile 11 months or more U R entitled...,"[had, your, mobile, 11, months, or, more, u, r...","[mobile, 11, months, u, r, entitled, update, l..."


In [13]:
#Stemming
ps = nltk.PorterStemmer()

def stemming(tokenized_text):
    stemmed_text = [ps.stem(word) for word in tokenized_text]
    return stemmed_text

df['stemmed_text'] = df.clean_text.apply(lambda row : stemming(row))
df[['text','tokenized_text','stemmed_text']].head()

,text,tokenized_text,stemmed_text
0,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazi, avail, bugi, n, gre..."
1,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entri, 2, wkli, comp, win, fa, cup, fin..."
3,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, earli, hor, u, c, alreadi, say]"
4,"Nah I don't think he goes to usf, he lives aro...","[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goe, usf, live, around, tho..."


In [14]:
def final_text(stemmed_text):
    get_final_text = " ".join([word for word in stemmed_text])
    return get_final_text

df['final_text'] = df.stemmed_text.apply(lambda row : final_text(row))
df.head(10)

,label,text,spam,length,no_punc,tokenized_text,clean_text,stemmed_text,final_text
0,ham,"Go until jurong point, crazy.. Available only ...",0,111,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...","[go, jurong, point, crazi, avail, bugi, n, gre...",go jurong point crazi avail bugi n great world...
1,ham,Ok lar... Joking wif u oni...,0,29,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]","[ok, lar, joke, wif, u, oni]",ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,155,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...",free entri 2 wkli comp win fa cup final tkt 21...
3,ham,U dun say so early hor... U c already then say...,0,49,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]","[u, dun, say, earli, hor, u, c, alreadi, say]",u dun say earli hor u c alreadi say
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,61,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...",nah dont think goe usf live around though
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1,147,FreeMsg Hey there darling its been 3 weeks now...,"[freemsg, hey, there, darling, its, been, 3, w...","[freemsg, hey, darling, 3, weeks, word, back, ...","[freemsg, hey, darl, 3, week, word, back, id, ...",freemsg hey darl 3 week word back id like fun ...
6,ham,Even my brother is not like to speak with me. ...,0,77,Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids...","[even, brother, like, speak, treat, like, aid,...",even brother like speak treat like aid patent
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0,160,As per your request Melle Melle Oru Minnaminun...,"[as, per, your, request, melle, melle, oru, mi...","[per, request, melle, melle, oru, minnaminungi...","[per, request, mell, mell, oru, minnaminungint...",per request mell mell oru minnaminungint nurun...
8,spam,WINNER!! As a valued network customer you have...,1,157,WINNER As a valued network customer you have b...,"[winner, as, a, valued, network, customer, you...","[winner, valued, network, customer, selected, ...","[winner, valu, network, custom, select, receiv...",winner valu network custom select receivea 900...
9,spam,Had your mobile 11 months or more? U R entitle...,1,154,Had your mobile 11 months or more U R entitled...,"[had, your, mobile, 11, months, or, more, u, r...","[mobile, 11, months, u, r, entitled, update, l...","[mobil, 11, month, u, r, entitl, updat, latest...",mobil 11 month u r entitl updat latest colour ...


In [15]:
#Comparing the Difference between original text and final text i.e Before & After text pre-processing
df[['text', 'final_text']].head(10)

,text,final_text
0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,"Nah I don't think he goes to usf, he lives aro...",nah dont think goe usf live around though
5,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darl 3 week word back id like fun ...
6,Even my brother is not like to speak with me. ...,even brother like speak treat like aid patent
7,As per your request 'Melle Melle (Oru Minnamin...,per request mell mell oru minnaminungint nurun...
8,WINNER!! As a valued network customer you have...,winner valu network custom select receivea 900...
9,Had your mobile 11 months or more? U R entitle...,mobil 11 month u r entitl updat latest colour ...


In [16]:
sms = df['final_text']
print(sms)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri 2 wkli comp win fa cup final tkt 21...
3                     u dun say earli hor u c alreadi say
4               nah dont think goe usf live around though
                              ...                        
5567    2nd time tri 2 contact u u 750 pound prize 2 c...
5568                                b go esplanad fr home
5569                              piti mood soani suggest
5570    guy bitch act like id interest buy someth els ...
5571                                       rofl true name
Name: final_text, Length: 5572, dtype: object


In [18]:
#CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

sms = df['final_text']
vectorizer = CountVectorizer()
vectorizer.fit(sms)
#print(vectorizer.vocabulary_)

vec = vectorizer.transform(sms)
# print(vec.shape)
# print(vec.toarray())

NameError: name 'sklearn' is not defined

In [18]:
#Verifying the function
vec

<5572x7976 sparse matrix of type '<class 'numpy.int64'>'
	with 46847 stored elements in Compressed Sparse Row format>

In [19]:
vec.shape[0]

5572

In [20]:
vec.shape[1]

7976

In [21]:
#TF-IDF 
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(vec)

data_tfidf = tfidf_transformer.transform(vec)
data_tfidf

<5572x7976 sparse matrix of type '<class 'numpy.float64'>'
	with 46847 stored elements in Compressed Sparse Row format>

In [22]:
#Splitting the data in Training and Testing Set
from sklearn.model_selection import train_test_split
data_tfidf_train, data_tfidf_test, label_train, label_test = \
     train_test_split(data_tfidf, df['spam'], test_size=0.1, random_state=1 )

In [23]:
data_tfidf_train

<5014x7976 sparse matrix of type '<class 'numpy.float64'>'
	with 42111 stored elements in Compressed Sparse Row format>

In [24]:
data_tfidf_test

<558x7976 sparse matrix of type '<class 'numpy.float64'>'
	with 4736 stored elements in Compressed Sparse Row format>

In [43]:
import pickle 
pickle.dump(svm,open('svm.pkl','wb'))

In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics 

In [48]:
#Applying Multinomial Naive Bayes Algo

nb = MultinomialNB()
nb.fit(data_tfidf_train, label_train)
y_pred_class = nb.predict(data_tfidf_test)
score = metrics.accuracy_score(label_test, y_pred_class)
score1 = metrics.recall_score(label_test, y_pred_class)
precision = metrics.precision_score(label_test, y_pred_class)
Accuracy = score * 100
print(Accuracy,'%')
print(score1)
print(precision)
cf_matrix=metrics.confusion_matrix(label_test, y_pred_class)
cf_matrix

97.4910394265233 %
0.7941176470588235
1.0


array([[490,   0],
       [ 14,  54]], dtype=int64)

In [52]:
##Applying SVC Algo

svm = SVC()
svm.fit(data_tfidf_train, label_train)
y_pred_class = svm.predict(data_tfidf_test)
score = metrics.accuracy_score(label_test, y_pred_class)
score1 = metrics.recall_score(label_test, y_pred_class)
precision = metrics.precision_score(label_test, y_pred_class)
f1 = metrics.f1_score(label_test, y_pred_class)
Accuracy = score * 100
print(Accuracy,'%')
print(score1)
print(precision)
print(f1)
cf_matrix=metrics.confusion_matrix(label_test, y_pred_class)
cf_matrix

98.56630824372759 %
0.8823529411764706
1.0
0.9375


array([[490,   0],
       [  8,  60]], dtype=int64)

In [28]:
##Applying K-Nearest Neighbour Algo

knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn.fit(data_tfidf_train, label_train)
y_pred_class = knn.predict(data_tfidf_test)
score = metrics.accuracy_score(label_test, y_pred_class)
Accuracy = score * 100
print(Accuracy,'%')
cf_matrix=metrics.confusion_matrix(label_test, y_pred_class)
cf_matrix

92.47311827956989 %


array([[490,   0],
       [ 42,  26]], dtype=int64)

In [29]:
#Applying Random Forest Algo

rf = RandomForestClassifier(n_estimators = 40)
rf.fit(data_tfidf_train, label_train)
y_pred_class = rf.predict(data_tfidf_test)
score = metrics.accuracy_score(label_test, y_pred_class)
Accuracy = score * 100
print(Accuracy,'%')
cf_matrix=metrics.confusion_matrix(label_test, y_pred_class)
cf_matrix

98.38709677419355 %


array([[488,   2],
       [  7,  61]], dtype=int64)

In [51]:

dt = tree.DecisionTreeClassifier()
dt.fit(data_tfidf_train, label_train)
y_pred_class = dt.predict(data_tfidf_test)
score = metrics.accuracy_score(label_test, y_pred_class)
score1 = metrics.recall_score(label_test, y_pred_class)
precision = metrics.precision_score(label_test, y_pred_class)
f1 = metrics.f1_score(label_test, y_pred_class)
Accuracy = score * 100
print(Accuracy,'%')
print(score1)
print(precision)
print(f1)
cf_matrix=metrics.confusion_matrix(label_test, y_pred_class)
cf_matrix

96.415770609319 %
0.8529411764705882
0.8529411764705882
0.8529411764705882


array([[480,  10],
       [ 10,  58]], dtype=int64)

In [35]:
def classifier():
    msg = input("Enter a message : ")
    msg = msg.lower()
    msg = rm_punc(msg)
    msg = tok(msg)
    msg = rm_sw(msg)
    msg = stemming(msg)
    msg = final_text(msg)
    msg = vectorizer.transform([msg])
    msg = tfidf_transformer.transform(msg)
    msg = svm.predict(msg)
    if msg == [1]:
        print("-------------SPAM!!--------------")
    else:
        print("-------------Ham-------------")

In [67]:
classifier(msg)

Enter a message : Had your contract mobile 11 Mnths? Latest Motorola, Nokia etc. all FREE! Double Mins & Text on Orange tariffs. TEXT YES for callback, no to remove from records.
-------------SPAM!!--------------


In [68]:
classifier(msg)

Enter a message : Free entry in 2 a wkly comp to win FA Cup final...
-------------SPAM!!--------------


In [69]:
classifier(msg)

Enter a message : Hello? Is anyone here?
-------------Ham-------------


In [77]:
classifier(msg)

Enter a message : Free entry in 2 a weekly competition to win FA cup Final
-------------SPAM!!--------------


In [78]:
classifier(msg)

Enter a message : congratulations Rs. 5500 in your rummy account play rummy and win prizes register for free uv8.me/pea9so
-------------SPAM!!--------------


In [37]:
classifier()

Enter a message : congratulations Rs. 5500 in your rummy account play rummy and win prizes register for free uv8.me/pea9so
-------------SPAM!!--------------


In [39]:
classifier()

Enter a message : Had your contract mobile 11 Mnths? Latest Motorola, Nokia etc. all FREE! Double Mins & Text on Orange tariffs. TEXT YES for callback, no to remove from records.
-------------SPAM!!--------------


In [40]:
classifier()

Enter a message : Free entry in 2 a weekly competition to win FA cup Final
-------------SPAM!!--------------


In [44]:
classifier()

Enter a message : Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed å£1000 cash or å£5000 prize!
-------------SPAM!!--------------


In [59]:
classifier()

Enter a message : congratulations Rs. 5500 in your rummy account play rummy and win prizes register for free uv8.me/pea9so
-------------SPAM!!--------------
